In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

In [2]:
rml_file_path = './data/psg_audio/rml_clean/00000995-100507.rml'

In [3]:
tree = ET.parse(rml_file_path)
root = tree.getroot()

In [4]:
print(root.tag, root.attrib.get('Version'))

{http://www.respironics.com/PatientStudy.xsd}PatientStudy 6


In [5]:
taglists = []

for child in root:
    taglists.append(child.tag.split('}')[1]) 

taglists

['Patient',
 'ChannelConfig',
 'Acquisition',
 'CustomEventTypeDefs',
 'AcquisitionCommentDefs',
 'ScoringData',
 'AnalysisOptions',
 'ReportCalcOptions',
 'EventFilters',
 'TrendChannelMappings',
 'BodyPositionState']

In [6]:
for child in root[1]:
    print(child.tag, child.text)

{http://www.respironics.com/PatientStudy.xsd}ConfigName LDxS_PSG_AASM-C3-C4
{http://www.respironics.com/PatientStudy.xsd}Channels 
         


In [7]:
ns = { 'study' : 'http://www.respironics.com/PatientStudy.xsd'}

for patient in root.findall('study:Patient', ns):
    gender = patient.find('study:Gender', ns)
    print(gender.text)

Female


In [8]:
for AcqNumber in root.findall('study:Acquisition', ns):
    number = AcqNumber.find('study:AcqNumber', ns)
    print(number.text)

995


In [9]:
ns = { 'study' : 'http://www.respironics.com/PatientStudy.xsd'}

acq = root.findall('study:Acquisition', ns)

acq_number = [x.find('study:AcqNumber', ns).text for x in acq]

print(acq_number)

['995']


In [10]:
root.find('study:Acquisition', ns).find('study:Sessions', ns).find('study:Session', ns).find('study:Segments', ns)

<Element '{http://www.respironics.com/PatientStudy.xsd}Segments' at 0x000001BAF0F70860>

In [11]:
segment = root.find('study:Acquisition', ns).find('study:Sessions', ns).find('study:Session', ns).find('study:Segments', ns)

duration = [x.find('study:Duration', ns).text for x in segment]

print(duration)

['3600', '3600', '3600', '3600', '3491']


In [12]:
stages = root.find('study:ScoringData', ns).find('study:StagingData', ns).find('study:UserStaging', ns).find('study:NeuroAdultAASMStaging', ns).findall('study:Stage', ns)

all_sleep = [x.attrib for x in stages]

for i in range(len(all_sleep)):
    if i < len(all_sleep) - 1 :
        all_sleep[i]['End'] = int(all_sleep[i + 1]['Start'])
    else:
        all_sleep[i]['End'] = int(all_sleep[i]['Start'])

wake = [x for x in all_sleep if x.get('Type') == 'Wake']

rem = [x for x in all_sleep if x.get('Type') == 'REM']

del all_sleep[-1]


In [13]:
all_sleep

[{'Type': 'Wake', 'Start': '0', 'End': 210},
 {'Type': 'NonREM1', 'Start': '210', 'End': 300},
 {'Type': 'NonREM2', 'Start': '300', 'End': 870},
 {'Type': 'Wake', 'Start': '870', 'End': 930},
 {'Type': 'NonREM1', 'Start': '930', 'End': 960},
 {'Type': 'NonREM2', 'Start': '960', 'End': 1920},
 {'Type': 'Wake', 'Start': '1920', 'End': 1980},
 {'Type': 'NonREM1', 'Start': '1980', 'End': 2010},
 {'Type': 'NonREM2', 'Start': '2010', 'End': 3300},
 {'Type': 'NonREM1', 'Start': '3300', 'End': 3330},
 {'Type': 'Wake', 'Start': '3330', 'End': 3360},
 {'Type': 'NonREM1', 'Start': '3360', 'End': 3420},
 {'Type': 'NonREM2', 'Start': '3420', 'End': 5220},
 {'Type': 'NonREM3', 'Start': '5220', 'End': 5400},
 {'Type': 'NonREM2', 'Start': '5400', 'End': 5460},
 {'Type': 'NonREM3', 'Start': '5460', 'End': 5520},
 {'Type': 'NonREM2', 'Start': '5520', 'End': 5850},
 {'Type': 'NonREM3', 'Start': '5850', 'End': 6120},
 {'Type': 'NonREM2', 'Start': '6120', 'End': 6540},
 {'Type': 'NonREM3', 'Start': '6540',

In [14]:
events = root.find('study:ScoringData', ns).find('study:Events', ns).findall('study:Event', ns)

respiratory = [x.attrib for x in events if x.attrib.get('Family') == 'Respiratory']

for i in range(len(respiratory)):
    respiratory[i]['End'] = float(respiratory[i]['Start']) + float(respiratory[i]['Duration'])

In [15]:
type(respiratory[0])

dict

In [16]:
# respiratory[0]['End'] = float(respiratory[0]['Start']) + float(respiratory[0]['Duration'])

In [17]:
for i in range(len(respiratory)):
    respiratory[i]['End'] = float(respiratory[i]['Start']) + float(respiratory[i]['Duration'])

In [18]:
type(respiratory)

list

In [19]:
type(wake[0]['Start'])

str

In [20]:
import pandas as pd

In [21]:
# wake_df = pd.DataFrame.from_dict(wake)
# rem_df = pd.DataFrame.from_dict(rem)

# stages_df = pd.concat([wake_df, rem_df], axis=0).astype({'Start' : 'int' , 'End' : 'int'}).sort_values(by='Start').reset_index(drop=True)

# stages_df['patient_id'] = 0
# patient_id = {0 : acq_number}

# stages_df['patient_id'] = stages_df['patient_id'].replace(patient_id)

# breath_df = pd.DataFrame.from_dict(respiratory).astype({'Start' : 'float' , 'End' : 'float'}).sort_values(by='Start').reset_index(drop=True).drop('Family', axis=1)

In [22]:
stages_df = pd.DataFrame.from_dict(all_sleep).astype({'Start' : 'int' , 'End' : 'int'}).sort_values(by='Start').reset_index(drop=True)

# stages_df['patient_id'] = 0
# patient_id = {0 : acq_number}

# stages_df['patient_id'] = stages_df['patient_id'].replace(patient_id)
stages_df['stage_duration'] = stages_df['End'] - stages_df['Start']
stages_df['stage_duration'] = stages_df['stage_duration'].astype('float')

breath_df = pd.DataFrame.from_dict(respiratory).astype({'Start' : 'float' , 'End' : 'float', 'Duration' : 'float'}).sort_values(by='Start').reset_index(drop=True).drop('Family', axis=1)

In [23]:
stages_df.rename(columns={'Type' : 'stage_type', 'Start' : 'stage_start_time', 'End' : 'stage_end_time' , 'Duration' : 'stage_duration'}, inplace=True)
breath_df.rename(columns={'Start' : 'event_start_time', 'End' : 'event_end_time' , 'Duration' : 'event_duration', 'Type' : 'respiratory_type'}, inplace=True)

In [31]:
stages_df['patient_id'] = np.zeros(len(stages_df))

In [32]:
stages_df

,stage_type,stage_start_time,stage_end_time,stage_duration,patient_id
0,Wake,0,210,210.0,0.0
1,NonREM1,210,300,90.0,0.0
2,NonREM2,300,870,570.0,0.0
3,Wake,870,930,60.0,0.0
4,NonREM1,930,960,30.0,0.0
5,NonREM2,960,1920,960.0,0.0
6,Wake,1920,1980,60.0,0.0
7,NonREM1,1980,2010,30.0,0.0
8,NonREM2,2010,3300,1290.0,0.0
9,NonREM1,3300,3330,30.0,0.0


In [24]:
test_list = {'a':1}
add_list = {'b':2}
for key in add_list.keys():
    test_list[key] = add_list.get(key)

test_list

{'a': 1, 'b': 2}

In [25]:
def get_matched_df(df, breathing_df):
    matched_df = pd.DataFrame(columns = ['patient_id', 'stage_start_time', 'stage_end_time', 'stage_duration', 'stage_type', 'event_start_time', 'event_end_time', 'event_duration', 'respiratory_type'])
    for i in range(len(df)):
        stage_start_time = df['stage_start_time'][i]
        stage_end_time = df['stage_end_time'][i]
        stage_duration = df['stage_duration'][i]
        stage_type = df['stage_type'][i]
        patient_id = df['patient_id'][i]
        event_start_time = []
        event_end_time = []
        event_duration = []
        respiratory_type = []
        for j in range(len(breathing_df)):
            if stage_start_time <= breathing_df['event_start_time'][j] and breathing_df['event_end_time'][j] <= stage_end_time:
                event_start_time.append(breathing_df['event_start_time'][j])
                event_end_time.append(breathing_df['event_end_time'][j])
                event_duration.append(breathing_df['event_duration'][j])
                respiratory_type.append(breathing_df['respiratory_type'][j])
        if len(event_start_time) == 0:
            matched_df = matched_df.append({'patient_id' : patient_id, 'stage_start_time' : stage_start_time, 'stage_end_time' : stage_end_time, 'stage_duration' : stage_duration, 'stage_type' : stage_type, 'event_start_time' : 'NaN', 'event_end_time' : 'NaN', 'event_duration' : 'NaN', 'respiratory_type' : 'NaN'}, ignore_index=True)
        else:
            for k in range(len(event_start_time)):
                matched_df = matched_df.append({'patient_id' : patient_id, 'stage_start_time' : stage_start_time, 'stage_end_time' : stage_end_time, 'stage_duration' : stage_duration, 'stage_type' : stage_type, 'event_start_time' : event_start_time[k], 'event_end_time' : event_end_time[k], 'event_duration' : event_duration[k], 'respiratory_type' : respiratory_type[k]}, ignore_index=True)
    return matched_df

In [26]:
def get_matched_df(self):
    sleep_df = self.get_df()
    breathing_df = self.get_breathing_df()

    matched_df = []
    for i in range(len(breathing_df)):
        # Find the sleep stages that overlap with the current breathing event
        overlapping_stages = sleep_df[(sleep_df['stage_end_time'] > breathing_df['event_start_time'][i]) & (sleep_df['stage_start_time'] < breathing_df['event_end_time'][i])]
        if len(overlapping_stages) > 0:
            # If there are overlapping sleep stages, create a row in the matched_df for each one
            for j in range(len(overlapping_stages)):
                row = {
                    'patient_id': overlapping_stages['patient_id'].iloc[j],
                    'stage_start_time': overlapping_stages['stage_start_time'].iloc[j],
                    'stage_end_time': overlapping_stages['stage_end_time'].iloc[j],
                    'stage_duration': overlapping_stages['stage_duration'].iloc[j],
                    'stage_type': overlapping_stages['stage_type'].iloc[j],
                    'event_start_time': breathing_df['event_start_time'][i],
                    'event_end_time': breathing_df['event_end_time'][i],
                    'event_duration': breathing_df['event_duration'][i],
                    'respiratory_type': breathing_df['respiratory_type'][i]
                }
                matched_df.append(row)
        else:
            # If there are no overlapping breathing events, create a row in the matched_df with NaN values for the event-related columns
            row = {
                'patient_id': sleep_df['patient_id'].iloc[i],
                'stage_start_time': sleep_df['stage_start_time'].iloc[i],
                'stage_end_time': sleep_df['stage_end_time'].iloc[i],
                'stage_duration': sleep_df['stage_duration'].iloc[i],
                'stage_type': sleep_df['stage_type'].iloc[i],
                'event_start_time': np.nan,
                'event_end_time': np.nan,
                'event_duration': np.nan,
                'respiratory_type': np.nan
            }
            matched_df.append(row)

    matched_df = pd.DataFrame(matched_df).sort_values(by= 'stage_start_time').reset_index(drop=True)
    return matched_df

In [27]:
def get_matched_df2(df, breathing_df):
    matched_df = pd.DataFrame(columns = ['patient_id', 'stage_start_time', 'stage_end_time', 'stage_duration', 'stage_type', 'event_start_time', 'event_end_time', 'event_duration', 'respiratory_type'])
    for i in range(len(df)):
        stage_start_time = df['stage_start_time'][i]
        stage_end_time = df['stage_end_time'][i]
        stage_duration = df['stage_duration'][i]
        stage_type = df['stage_type'][i]
        patient_id = df['patient_id'][i]
        event_start_time = []
        event_end_time = []
        event_duration = []
        respiratory_type = []
        for j in range(len(breathing_df)):
            if stage_start_time <= breathing_df['event_start_time'][j] and breathing_df['event_end_time'][j] <= stage_end_time:
                event_start_time.append(breathing_df['event_start_time'][j])
                event_end_time.append(breathing_df['event_end_time'][j])
                event_duration.append(breathing_df['event_duration'][j])
                respiratory_type.append(breathing_df['respiratory_type'][j])
        if len(event_start_time) == 0:
            matched_df = pd.concat([matched_df, pd.DataFrame({'patient_id' : patient_id, 'stage_start_time' : stage_start_time, 'stage_end_time' : stage_end_time, 'stage_duration' : stage_duration, 'stage_type' : stage_type, 'event_start_time' : 'NaN', 'event_end_time' : 'NaN', 'event_duration' : 'NaN', 'respiratory_type' : 'NaN'}, index=[0])], ignore_index=True)
        else:
            for k in range(len(event_start_time)):
                matched_df = pd.concat([matched_df, pd.DataFrame({'patient_id' : patient_id, 'stage_start_time' : stage_start_time, 'stage_end_time' : stage_end_time, 'stage_duration' : stage_duration, 'stage_type' : stage_type, 'event_start_time' : event_start_time[k], 'event_end_time' : event_end_time[k], 'event_duration' : event_duration[k], 'respiratory_type' : respiratory_type[k]}, index=[0])], ignore_index=True)
    return matched_df
 

In [28]:
new2 = get_matched_df2(stages_df, breath_df)
new2

KeyError: 'patient_id'

,patient_id,stage_start_time,stage_end_time,stage_duration,stage_type,event_start_time,event_end_time,event_duration,respiratory_type
0,995,0,210,210.0,Wake,NaN,NaN,NaN,NaN
1,995,210,300,90.0,NonREM1,NaN,NaN,NaN,NaN
2,995,300,870,570.0,NonREM2,NaN,NaN,NaN,NaN
3,995,870,930,60.0,Wake,NaN,NaN,NaN,NaN
4,995,930,960,30.0,NonREM1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
234,995,17400,17700,300.0,REM,17642.5,17658.5,16.0,ObstructiveApnea
235,995,17700,17730,30.0,Wake,NaN,NaN,NaN,NaN
236,995,17730,17760,30.0,NonREM1,NaN,NaN,NaN,NaN
237,995,17760,17880,120.0,NonREM2,17820.5,17832.0,11.5,Hypopnea


In [ ]:
stages_df

,stage_type,stage_start_time,stage_end_time,patient_id,stage_duration
0,Wake,0,210,995,210.0
1,NonREM1,210,300,995,90.0
2,NonREM2,300,870,995,570.0
3,Wake,870,930,995,60.0
4,NonREM1,930,960,995,30.0
5,NonREM2,960,1920,995,960.0
6,Wake,1920,1980,995,60.0
7,NonREM1,1980,2010,995,30.0
8,NonREM2,2010,3300,995,1290.0
9,NonREM1,3300,3330,995,30.0


In [ ]:
new.to_csv('995_2.csv', index=False)

In [ ]:
print(breath_df.loc[breath_df['Type'] == 'ObstructiveApnea'].Duration.max(),
breath_df.loc[breath_df['Type'] == 'MixedApnea'].Duration.max())

KeyError: 'Type'

In [ ]:
breath_df.loc[breath_df['Type'] == 'Hypopnea'].loc[breath_df['Duration'] == 18]


,Type,Start,Duration,End
71,Hypopnea,9104.5,18.0,9122.5


In [ ]:
breath_df.loc[breath_df['Type'] == 'ObstructiveApnea'].loc[breath_df['Duration'] == 21]

,Type,Start,Duration,End
205,ObstructiveApnea,17539.5,21.0,17560.5


In [ ]:
breath_df.loc[breath_df['Type'] == 'MixedApnea'].loc[breath_df['Duration'] == 17.5]

,Type,Start,Duration,End
192,MixedApnea,16848.5,17.5,16866.0


In [ ]:
print(9104 / 3600, 17539.5 / 3600, 16848.5 / 3600)

2.528888888888889 4.872083333333333 4.680138888888889
